# Extractor

In [2]:
import PyPDF2
import csv
import re
from pathlib import Path
import pandas as pd

class EUCSExtractor:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.controls = []
        
    def extract_text_from_pdf(self):
        """Estrae il testo dal PDF"""
        text = ""
        try:
            with open(self.pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                print(f"📄 PDF aperto: {len(pdf_reader.pages)} pagine")
                for page_num, page in enumerate(pdf_reader.pages, 1):
                    text += page.extract_text()
            return text
        except Exception as e:
            print(f"❌ Errore nella lettura del PDF: {e}")
            return None
    
    def parse_controls(self, text):
        """Estrae i controlli dal testo con pattern matching"""
        if not text:
            return
        
        lines = text.split('\n')
        
        current_control = {
            'Ref': '',
            'Description': '',
            'Ass. Level': ''
        }
        in_control_section = False
        description_lines = []
        
        # Pattern per riconoscere nuove sezioni (es: "OIS-02 SEGREGATION OF DUTIES")
        section_header_pattern = r'^[A-Z]{2,3}-\d{2}\s+[A-Z\s]+'
        # Pattern per Ref: e.g., "OIS-01.1", "HR-04.7"
        ref_pattern = r'^([A-Z]{2,3}-\d{2}\.\d{1,2})\s+'
        # Levels: Basic, Substantial, High
        level_pattern = r'\b(Basic|Substantial|High)\s*$'
        
        for i, line in enumerate(lines):
            line_stripped = line.strip()
            
            # Riconosciamo l'inizio della sezione Requirements
            if 'Ref' in line_stripped and 'Description' in line_stripped and 'Ass. Level' in line_stripped:
                in_control_section = True
                continue
            
            if not in_control_section or not line_stripped:
                continue
            
            # Se raggiungiamo una sezione Guidance, saltiamo (o usciamo dalla sezione controlli)
            if line_stripped.startswith('Guidance'):
                in_control_section = False
                continue
            
            # Se raggiungiamo una nuova sezione (header), salviamo il controllo attuale e ricominciamo
            if re.match(section_header_pattern, line_stripped) and current_control['Ref']:
                if current_control['Ref'] and description_lines:
                    self.controls.append({
                        'Ref': current_control['Ref'],
                        'Description': ' '.join(description_lines).strip(),
                        'Ass. Level': current_control['Ass. Level'].strip()
                    })
                current_control = {'Ref': '', 'Description': '', 'Ass. Level': ''}
                description_lines = []
                # Nota: qui potresti voler mantenere in_control_section = True se le sezioni sono contigue
                continue
            
            match = re.match(ref_pattern, line_stripped)
            
            if match:
                # Trovato un nuovo codice controllo (es. OIS-01.1)
                
                # Salviamo il controllo precedente se esiste
                if current_control['Ref'] and description_lines:
                    self.controls.append({
                        'Ref': current_control['Ref'],
                        'Description': ' '.join(description_lines).strip(),
                        'Ass. Level': current_control['Ass. Level'].strip()
                    })
                
                # Iniziamo un nuovo controllo
                current_control['Ref'] = match.group(1)
                description_lines = []
                current_control['Ass. Level'] = ''
                
                # Estraiamo il resto della linea dopo il Ref
                rest_of_line = line_stripped[len(match.group(1)):].strip()
                
                # Cerchiamo il livello di assessment alla fine della linea
                level_match = re.search(level_pattern, rest_of_line)
                
                if level_match:
                    current_control['Ass. Level'] = level_match.group(1)
                    description_text = rest_of_line[:level_match.start()].strip()
                else:
                    description_text = rest_of_line
                
                if description_text:
                    description_lines.append(description_text)
            
            elif current_control['Ref'] and line_stripped:
                # È una riga di continuazione della descrizione
                
                # Verifichiamo se il livello di assessment è su questa riga (a volte va a capo)
                level_match = re.search(level_pattern, line_stripped)
                
                if level_match and not current_control['Ass. Level']:
                    current_control['Ass. Level'] = level_match.group(1)
                    description_text = line_stripped[:level_match.start()].strip()
                else:
                    description_text = line_stripped
                
                if description_text:
                    description_lines.append(description_text)
        
        # Salviamo l'ultimo controllo rimasto in memoria alla fine del ciclo
        if current_control['Ref'] and description_lines:
            self.controls.append({
                'Ref': current_control['Ref'],
                'Description': ' '.join(description_lines).strip(),
                'Ass. Level': current_control['Ass. Level'].strip()
            })

    def save_to_csv(self, output_path):
        """Salva i controlli estratti in un file CSV"""
        try:
            with open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
                fieldnames = ['Ref', 'Description', 'Ass. Level']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                
                writer.writeheader()
                writer.writerows(self.controls)
            
            print(f"✅ CSV salvato con successo: {output_path}")
            print(f"📊 Totale controlli estratti: {len(self.controls)}")
        except Exception as e:
            print(f"❌ Errore nel salvataggio del CSV: {e}")
    
    def extract(self, output_path):
        """Esegue l'estrazione completa"""
        print("🚀 Inizio estrazione EUCS PDF...")
        text = self.extract_text_from_pdf()
        if text:
            print("✅ PDF letto correttamente")
            self.parse_controls(text)
            print(f"✅ {len(self.controls)} controlli trovati")
            self.save_to_csv(output_path)
            return self.controls
        else:
            print("❌ Impossibile leggere il PDF")
            return None
    
    def display_preview(self, n=5):
        """Mostra anteprima dei primi n controlli"""
        if not self.controls:
            print("❌ Nessun controllo da visualizzare")
            return
        
        df = pd.DataFrame(self.controls[:n])
        print(f"\n📋 Anteprima dei primi {n} controlli:")
        print(df.to_string(index=False))


# ============ ESECUZIONE NOTEBOOK ============

# Assicurati che il percorso del file sia corretto
pdf_path = "Other/EUCS PDF official file/EUCS_Pagine_85-159.pdf"
output_csv = "Other/EUCS PDF official file/eucs_controls.csv"

extractor = EUCSExtractor(pdf_path)
controls = extractor.extract(output_csv)

# Mostra anteprima
if controls:
    extractor.display_preview(10)
    
    # Crea un DataFrame per ulteriori analisi
    df = pd.DataFrame(controls)
    print(f"\n📈 Statistiche:")
    print(f"Total Controls: {len(df)}")
    print(f"\nDistribuzione per livello di assessment:")
    print(df['Ass. Level'].value_counts())

🚀 Inizio estrazione EUCS PDF...
📄 PDF aperto: 75 pagine
✅ PDF letto correttamente
✅ 388 controlli trovati
✅ CSV salvato con successo: Other/EUCS PDF official file/eucs_controls.csv
📊 Totale controlli estratti: 388

📋 Anteprima dei primi 10 controlli:
     Ref                                                                                                                                                                                                                                                                                                                                                                                                                Description  Ass. Level
OIS-01.1                                                                                                                                                                                                     The CSP shall define,  implement, maintain and continually improve an information security management system (IS